## Simple PromptTemplate examples using langchain framework
- Examples with the different style of prompts in the context window
- Examples showing how to control the tone of the output or completion
- I have used ChatGPT's gpt-3.5-turbo model to perform the completions
- I have used ChatPromptTemplate to get the output as JSON and used a output parser to format the response we recieve in a JSON type.

In [1]:
import os
import openai

In [2]:
from dotenv import load_dotenv, find_dotenv

In [3]:
_ = load_dotenv(find_dotenv())

#### We are going to use a user-define utility class to get the secret keys from the environment variables set. Kinldy refer to UDCUtils.py file for the definition.

In [4]:
from UDCUtils import UDCUtils
utils = UDCUtils()
openai.api_key = utils.get_openai_api_key()
#openai.api_key = os.environ['OPENAI_API_KEY']

In [5]:
import datetime

current_date = datetime.datetime.now()

target_date = datetime.datetime(2024, 6, 12)

if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [6]:
llm_model

'gpt-3.5-turbo'

In [7]:
def get_completion(prompt, model=llm_model):
    messages = [
        {
            "role":"user",
            "content":prompt
        }
    ]
    response = openai.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0.0
    )
    return response.choices[0].message.content

###### In above cell we have made use of temperature attribute, it is a configuration to set how precise or accurate you need the completion to be. 0 indicates nearly accurate response whereas a higher value like 1.0 indicates the output to be very creative ones

In [11]:
from pprint import pprint
response = get_completion("Provide me with the dumbest product review for a umbrella by an american citizen")
pprint(response)

('"This umbrella is the worst! I tried using it in a rainstorm and it got all '
 'wet and soggy. What a ripoff! I thought umbrellas were supposed to keep you '
 'dry, but this one just made everything worse. Do not recommend."')


In [12]:
customer_email = """
This umbrella is the worst! I tried using it in a rainstorm and it got all wet and soggy. \
What a ripoff! I thought umbrellas were supposed to keep you dry, \
but this one just made everything worse. Help me get a refund.
"""

##### Now we are going to ask chat gpt to format the above customer email in a particular tone using the instructions in prompt as below

In [13]:
response_style = """
American style \
in a calm and polite tone.
"""

In [14]:
prompt_message = f"""
Translate the text that is delimited by triple backticks \
into a style that is {response_style} \
text : ```{customer_email}```
"""

In [15]:
print(prompt_message)


Translate the text that is delimited by triple backticks into a style that is 
American style in a calm and polite tone.
 text : ```
This umbrella is the worst! I tried using it in a rainstorm and it got all wet and soggy. What a ripoff! I thought umbrellas were supposed to keep you dry, but this one just made everything worse. Help me get a refund.
```



In [16]:
response = get_completion(prompt_message)

In [17]:
pprint(response)

('I would like to kindly express my disappointment with this umbrella. I used '
 'it during a rainstorm and unfortunately, it became wet and soggy. I was '
 'under the impression that umbrellas were meant to keep you dry, but this one '
 'did not meet my expectations. I would greatly appreciate your assistance in '
 'obtaining a refund. Thank you.')


### Lets do the above work using LangChain framework

In [15]:
#!pip install langchain_community
#!pip install langchain

In [28]:
from langchain.chat_models import ChatOpenAI

In [29]:
chat = ChatOpenAI(temperature=0.0,
                 model=llm_model,
                 openai_api_key = openai.api_key)

/opt/anaconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


##### Note down that we are going to use prompt templates to style and format our prompts, prompt templates make it easier to give instructions when the length of the input becomes longer and harder to format using string functions

In [18]:
template_style = """\
Translate the provided text delimited by three backticks into a style that is {style}\
text : ```{text}
"""

In [19]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_style)

In [20]:
pprint(prompt_template.messages[0].prompt)

PromptTemplate(input_variables=['style', 'text'], template='Translate the provided text delimited by three backticks into a style that is {style}text : ```{text}\n')


In [21]:
# this will give us the inputs we need to provide when sending instruction to llm
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [23]:
#passing the values of input variables
customer_message = prompt_template.format_messages(text=customer_email,
                                                  style=response_style)

In [24]:
print(type(customer_message))

<class 'list'>


In [25]:
print(customer_message)

[HumanMessage(content='Translate the provided text delimited by three backticks into a style that is \nAmerican style in a calm and polite tone.\ntext : ```\nThis umbrella is the worst! I tried using it in a rainstorm and it got all wet and soggy. What a ripoff! I thought umbrellas were supposed to keep you dry, but this one just made everything worse. Help me get a refund.\n\n')]


In [30]:
customer_formatted_message = chat(customer_message)

/opt/anaconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [31]:
pprint(customer_formatted_message.content)

('I am sorry to say that I am quite disappointed with this umbrella. I used it '
 'during a rainstorm and it ended up getting wet and soggy, which was not what '
 'I expected. I always thought umbrellas were meant to keep you dry, but '
 'unfortunately, this one did not live up to that expectation. I would greatly '
 'appreciate your assistance in helping me obtain a refund for this product. '
 'Thank you for your understanding.')


#### Some fun code to get the reply as well from chatgpt

In [32]:
service_reply = """
You are a customer support executive. Suggest me a good reply to the customer query \
provided in the text delimited by three backticks and \
the style of the reply should be in a {style} \
text : ```{text}```
"""

In [33]:
prompt_template = ChatPromptTemplate.from_template(service_reply)

In [34]:
print(prompt_template.messages[0].prompt.input_variables)

['style', 'text']


In [35]:
service_style = """
Casual american style in a sarcastic tone.
"""

In [36]:
service_message = prompt_template.format_messages(style=service_style,
                                                 text=customer_formatted_message.content)

In [37]:
generated_service_response = chat(service_message)

In [38]:
pprint(generated_service_response.content)

("Well, gee, sorry to hear about your soggy umbrella experience. Who would've "
 'thought an umbrella would actually get wet in a rainstorm, right? I mean, '
 "that's just unheard of! Let me see what I can do to help you out with that "
 'refund. Thanks for sharing your unique insight on umbrella functionality.')


#### Well that was a funny response, right?
#### Lets get started with output parsers in LangChain where we would be expecting completion in json format, we will read this as a JSON using a parser and can use this dict object for further operations.

In [39]:
#this is a json
sample_json = {
    "gift":True,
    "delivery":"5 days",
    "price_value":"It is pretty affordable."
}

#### Again I am going to use chatgpt to generate a customer review that was purchased as a gift on birthday.

In [40]:
prompt="""
Suggest me a funny review about an expensive cosmetic product an American man purchased as a gift on his wife's birthday which did not make any sense.
"""

In [41]:
gpt_response = get_completion(prompt)

In [42]:
print(gpt_response)

"I bought this fancy schmancy face cream for my wife's birthday and let me tell you, it cost more than my monthly car payment. But did it make her look 10 years younger? Nope. Did it make her skin glow like a unicorn's? Not even close. All it did was make her smell like a bouquet of flowers threw up on her face. So if you're looking to waste your money on a product that makes zero sense, this is the one for you!"


In [43]:
customer_review = gpt_response

In [44]:
response_style = """
For the following text extract the below information:
gift: Was this item purchased as a gift for someone? Answer with True if yes, else False.
delivery_days: How many days it took for the item to get it delivered. If not known then answer with a -1.
price_value: Extract any sentence that mentions the price or value of the item bought. \
If not known then answer with value as Unknown. The values for this should be a comma separated Ptyhon list.

Format the output as JSON text with keys as:
gift
delivery_days
price_value

And the text is {text}
"""

In [45]:
json_promt_template = ChatPromptTemplate.from_template(response_style)
print(json_promt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='\nFor the following text extract the below information:\ngift: Was this item purchased as a gift for someone? Answer with True if yes, else False.\ndelivery_days: How many days it took for the item to get it delivered. If not known then answer with a -1.\nprice_value: Extract any sentence that mentions the price or value of the item bought. If not known then answer with value as Unknown. The values for this should be a comma separated Ptyhon list.\n\nFormat the output as JSON text with keys as:\ngift\ndelivery_days\nprice_value\n\nAnd the text is {text}\n'))]


In [46]:
print(json_promt_template.messages[0].prompt.input_variables)

['text']


In [47]:
json_product_review_prompt_message = json_promt_template.format_messages(text=customer_review)

In [48]:
json_formatted_output = chat(json_product_review_prompt_message)
print(json_formatted_output.content)

{
  "gift": true,
  "delivery_days": -1,
  "price_value": ["it cost more than my monthly car payment."]
}


In [50]:
pprint(json_formatted_output)

AIMessage(content='{\n  "gift": true,\n  "delivery_days": -1,\n  "price_value": ["it cost more than my monthly car payment."]\n}', response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 226, 'total_tokens': 257}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-56516a27-8697-4154-80de-7e08862aff20-0')


In [51]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [52]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was this item purchased as a gift for someone? Answer as True if yes and False if no or not known. Output should be of type boolean."
)

delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days it took for the item to get it delivered. If not known then answer with a -1."
)

price_val_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentence that mentions the price or value of the item bought. If not known then answer with value as Unknown. The values for this should be a comma separated Ptyhon list."
)

response_schemas = [gift_schema, delivery_days_schema, price_val_schema]

In [53]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

pprint(format_instructions)

('The output should be a markdown code snippet formatted in the following '
 'schema, including the leading and trailing "```json" and "```":\n'
 '\n'
 '```json\n'
 '{\n'
 '\t"gift": string  // Was this item purchased as a gift for someone? Answer '
 'as True if yes and False if no or not known. Output should be of type '
 'boolean.\n'
 '\t"delivery_days": string  // How many days it took for the item to get it '
 'delivered. If not known then answer with a -1.\n'
 '\t"price_value": string  // Extract any sentence that mentions the price or '
 'value of the item bought. If not known then answer with value as Unknown. '
 'The values for this should be a comma separated Ptyhon list.\n'
 '}\n'
 '```')


In [54]:
parser_response_style = """
For the following text extract the below information:
gift: Was this item purchased as a gift for someone? Answer as True if yes and False if no or not known. Output should be of type boolean.
delivery_days: How many days it took for the item to get it delivered. If not known then answer with a -1.
price_value: Extract any sentence that mentions the price or value of the item bought. \
If not known then answer with value as Unknown. The values for this should be a comma separated Ptyhon list.
text:{text}
{format_instructions}
"""

In [55]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [56]:
prompt_template = ChatPromptTemplate.from_template(parser_response_style)


prompt_message = prompt_template.format_messages(text=customer_review,
                                                format_instructions=format_instructions)

chat_0_2 = ChatOpenAI(model=llm_model, temperature=0.2, openai_api_key = openai.api_key)

json_output = chat_0_2(prompt_message)

print(type(json_output))

<class 'langchain_core.messages.ai.AIMessage'>


In [58]:
print(json_output.content)

```json
{
	"gift": true,
	"delivery_days": -1,
	"price_value": "it cost more than my monthly car payment"
}
```


In [59]:
print(type(json_output.content))

<class 'str'>


In [60]:
output_dict = output_parser.parse(json_output.content)

In [61]:
print(type(output_dict))

<class 'dict'>


In [62]:
print(output_dict.get("gift"))

True
